In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import numpy as np

In [16]:
per_pupil_exp = pd.read_excel('../data/per_pupil_exp_raw.xlsx')

In [17]:
per_pupil_exp = per_pupil_exp.iloc[22:1144]

In [20]:
per_pupil_exp = per_pupil_exp.drop(columns=['LocationType', 'DataFormat'])
per_pupil_exp['Schl_Year_Start'] = per_pupil_exp['TimeFrame'].str.split('-').str[0]
per_pupil_exp['Schl_Year_Start'] = per_pupil_exp['Schl_Year_Start'].astype(int)                                                         

KeyError: "['LocationType', 'DataFormat'] not found in axis"

In [22]:
per_pupil_exp.columns = ["State", "TimeFrame", "PPE", "Schl_Year_Start" ]

In [24]:
per_pupil_exp = per_pupil_exp[["State", "TimeFrame", "Schl_Year_Start", "PPE"]]

In [26]:
per_pupil_exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 22 to 1143
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   State            1122 non-null   object
 1   TimeFrame        1122 non-null   object
 2   Schl_Year_Start  1122 non-null   int64 
 3   PPE              1122 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 35.2+ KB


In [29]:
inflation_factors = {
    2000: 1.83, 2001: 1.76, 2002: 1.74, 2003: 1.70, 
    2004: 1.67, 2005: 1.62, 2006: 1.56, 2007: 1.52,
    2008: 1.46, 2009: 1.46, 2010: 1.42, 2011: 1.40,
    2012: 1.36, 2013: 1.34, 2014: 1.32, 2015: 1.32,
    2016: 1.30, 2017: 1.27, 2018: 1.24, 2019: 1.23,
    2020: 1.20, 2021: 1.18, 2022: 1.10, 2023: 1.03,
    2024: 1.00
}

per_pupil_exp["Inflation_Factor"] = per_pupil_exp["Schl_Year_Start"].map(inflation_factors)

per_pupil_exp["Value_adj"] = per_pupil_exp["PPE"] * per_pupil_exp["Inflation_Factor"]

per_pupil_exp["Value_adj"] = per_pupil_exp["Value_adj"].round(1)

C:\Users\mersi\AppData\Local\Temp\ipykernel_15164\3566669999.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_pupil_exp["Inflation_Factor"] = per_pupil_exp["Schl_Year_Start"].map(inflation_factors)
C:\Users\mersi\AppData\Local\Temp\ipykernel_15164\3566669999.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_pupil_exp["Value_adj"] = per_pupil_exp["PPE"] * per_pupil_exp["Inflation_Factor"]
C:\Users\mersi\AppData\Local\Temp\ipykernel_15164\3566669999.py:15: SettingWithCopyWarning: 
A value is tr

In [30]:
per_pupil_exp

,State,TimeFrame,Schl_Year_Start,PPE,Inflation_Factor,Value_adj
22,Alabama,2020-2021,2020,10728,1.20,12873.6
23,Alabama,2019-2020,2019,10140,1.23,12472.2
24,Alabama,2018-2019,2018,10107,1.24,12532.7
25,Alabama,2017-2018,2017,9717,1.27,12340.6
26,Alabama,2016-2017,2016,9528,1.30,12386.4
...,...,...,...,...,...,...
1139,Wyoming,2006-2007,2006,13266,1.56,20695.0
1140,Wyoming,2003-2004,2003,9308,1.70,15823.6
1141,Wyoming,2004-2005,2004,10190,1.67,17017.3
1142,Wyoming,2005-2006,2005,11437,1.62,18527.9


In [31]:
per_pupil_exp.to_csv("../data/per_pupil_exp_clean.csv", index=False)

In [32]:
pp_filtered = per_pupil_exp[
    (per_pupil_exp["Schl_Year_Start"] >= 2000) & 
    (per_pupil_exp["Schl_Year_Start"] <= 2021)
]


state_avg = (
    pp_filtered.groupby("State")["Value_adj"]
    .mean()
    .round(1)
    .reset_index()
)


state_avg_sorted = state_avg.sort_values(by="Value_adj", ascending=False).reset_index(drop=True)


In [33]:
state_avg_sorted

,State,Value_adj
0,District of Columbia,26382.3
1,New York,26081.2
2,New Jersey,24585.1
3,Connecticut,22963.9
4,Vermont,22627.7
5,Alaska,21604.2
6,Massachusetts,21033.2
7,Rhode Island,20546.6
8,Wyoming,19846.8
9,New Hampshire,18567.5


In [34]:
state_avg_sorted.to_csv("../data/avg_ppe_by_state.csv", index=False)